<a href="https://colab.research.google.com/github/zahraDehghanian97/LSTM_sequence_prediction/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **prerequisit**

In [9]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout , Masking
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import Sequence

# **load data**

In [12]:
# constant
num_class = 7736
dimension = 1
lstm_units = 200

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/predict_next_game/train_Copy.csv')
data = data.dropna()
X = []
y = []
seq_lens = []

for d in data.values:
    temp = []
    for t in d[1].split(" "):
        temp.append(int(t))
    X.append((temp))
    y.append( d[2])
    seq_lens.append(len(temp))
print(X)


[[3, 12, 262, 6094, 283, 50, 1070, 233], [294, 241, 1, 150, 12], [85, 139, 144, 57, 2013], [7, 114, 10, 5, 31, 6504], [5, 221, 3, 712, 159, 4, 810, 94, 746, 6170, 136, 17, 1160, 78, 113], [2, 1, 208, 1404, 1134, 413, 135, 40], [1022, 867, 26, 105, 631, 30, 664, 4, 48, 25, 25, 1015, 1398, 251, 257, 45, 3897, 2026, 88, 88], [8, 44, 1031, 54, 217], [42, 101, 5022, 3277, 6, 6, 197, 184, 762, 516, 773, 23, 18, 18, 420, 278, 122, 122, 652, 14], [1077, 1160, 648, 44, 5], [3634, 38, 51, 190, 127, 2422, 568, 5796, 578, 60, 1935], [5, 85, 180, 20, 286], [251, 3698, 1559, 41, 579, 144], [174, 469, 537, 1996, 269, 573, 2690, 1533, 2090, 1309], [7, 9, 27, 13, 10], [435, 1, 22, 230, 2494, 6554, 6585, 2070, 2, 52, 1781], [11, 68, 720, 12, 974, 4, 59, 478], [85, 2, 703, 60, 503, 6200, 107, 3535, 593, 132], [1, 84, 2391, 171, 4, 1140, 1212, 71, 71, 179], [373, 602, 2794, 901, 454, 2893, 315], [11, 48, 3, 81, 146, 5], [15, 625, 993, 474, 946, 947], [7, 9, 114, 10, 1338, 842], [168, 20, 483, 451, 54, 16,

# **Reshape data**

In [ ]:
# pad data with maximum len
special_value = -10.0
max_seq_len = max(seq_lens)
Xpad = np.full((len(seq_lens), max_seq_len, dimension), fill_value=special_value)
for s, x in enumerate(X):
    seq_len = len(x)
    xx = np.array(x)
    Xpad[s, 0:seq_len, :] = xx.reshape(seq_len,1)
    
y = np.array(y)

# **LSTM Model** : batch size = 1 

In [35]:
class MyBatchGenerator(Sequence):
    def __init__(self, X, y, batch_size=1, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.y) / self.batch_size))

    def __getitem__(self, index):
        return self.__data_generation(index)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.y))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        Xb = np.empty((self.batch_size, len(X[index])))
        yb = np.empty((self.batch_size,1))
        for s in range(0, self.batch_size):
            Xb[s] = X[index]
            yb[s] = y[index]
        print(index)
        print (Xb,yb)
        return Xb, yb


model = Sequential()
model.add(LSTM(lstm_units, return_sequences=True, input_shape=(None,dimension)))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(Dense(units=2 * num_class, activation='relu'))
model.add(Dense(units=num_class))
model.compile(optimizer='rmsprop', loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
print(model.summary())
model.fit_generator(MyBatchGenerator(X, y, batch_size=1), epochs=10)


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, None, 200)         161600    
_________________________________________________________________
dropout_44 (Dropout)         (None, None, 200)         0         
_________________________________________________________________
lstm_46 (LSTM)               (None, None, 200)         320800    
_________________________________________________________________
dropout_45 (Dropout)         (None, None, 200)         0         
_________________________________________________________________
lstm_47 (LSTM)               (None, None, 200)         320800    
_________________________________________________________________
dropout_46 (Dropout)         (None, None, 200)         0         
_________________________________________________________________
lstm_48 (LSTM)               (None, None, 200)       

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


ValueError: ignored

# **LSTM Model**  : padding

In [14]:

model = Sequential()
model.add(Masking(mask_value=special_value, input_shape=(max_seq_len, dimension)))
model.add(LSTM(lstm_units, return_sequences=True, input_shape=(None,dimension)))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dropout(rate=0.3))
model.add(Dense(units=2 * num_class, activation='relu'))
model.add(Dense(units=num_class, activation = 'sigmoid'))
model.compile(optimizer='rmsprop', loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
print(model.summary())
model.fit(Xpad, y, batch_size=32, epochs=50)



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_3 (Masking)          (None, 28, 1)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 28, 200)           161600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 28, 200)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 28, 200)           320800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 200)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 28, 200)           320800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 28, 200)          

InvalidArgumentError: ignored